### Libraries

In [0]:
import os
import re
import sys
import tarfile
import numpy as np
import math
from datetime import datetime
import time
from six.moves import urllib
import tensorflow as tf

### Set directories

In [0]:
data_dir = '/tmp/cifar10_data'
train_dir = '/tmp/cifar10_train'
eval_dir = '/tmp/cifar10_eval'

### Set scalars

In [0]:
train_size = 50000
test_size = 10000
batch_size = 128

new_size = 24

num_label = 10

max_step_train = 20000

max_step_eval = 10000

### Data link to download from

In [0]:
# Data link
DATA_URL = 'https://www.cs.toronto.edu/~kriz/cifar-10-binary.tar.gz'

### Download and extract data

In [0]:
if not os.path.exists(data_dir):
   os.makedirs(data_dir)  
filename = DATA_URL.split('/')[-1]
filepath = os.path.join(data_dir, filename)           
if not os.path.exists(filepath):
  def rh(count, block_size, total_size):
      sys.stdout.write('\r>> Downloading %s ' % filename)
      sys.stdout.flush()  
  filepath, _ = urllib.request.urlretrieve(DATA_URL, filepath, rh)
  print('\r>> Successfully downloaded', filename)
extracted_path = os.path.join(data_dir, 'cifar-10-batches-bin')
if not os.path.exists(extracted_path):
  tarfile.open(filepath, 'r:gz').extractall(data_dir)
print('file name:', filename)
print('file path:', filepath)
print('extracted path:', extracted_path)

file name: cifar-10-binary.tar.gz
file path: /tmp/cifar10_data/cifar-10-binary.tar.gz
extracted path: /tmp/cifar10_data/cifar-10-batches-bin


### Upload and decode cifar10

In [0]:
def decode_cifar10(filename_queue):
#Original image dimensions
    original_height = 32
    original_width = 32
    original_depth = 3

#Original Bytes
    label_bytes = 1 
    image_bytes = original_height * original_width * original_depth
    record_bytes = label_bytes + image_bytes


    reader = tf.FixedLengthRecordReader(record_bytes=record_bytes)
    key,value = reader.read(filename_queue)

    value_pixel = tf.decode_raw(value, tf.uint8)

    value_label = tf.cast(tf.strided_slice(value_pixel, [0], [label_bytes]), tf.int32)

    depth_major = tf.reshape(tf.strided_slice(value_pixel, 
                                          [label_bytes], 
                                          [label_bytes + image_bytes]),
                         [original_depth, original_height, original_width])

    image_unit8 = tf.transpose(depth_major, [1, 2, 0])
    
    return image_unit8, value_label

### Data Preprocess

In [0]:
def distort_data(filename_queue):
  with tf.name_scope('preprocessing'):
    image_decoded, label_decoded = decode_cifar10(filename_queue)
    reshaped_image = tf.cast(image_decoded, tf.float32)
    # Convert image data from unit8 to float32

    # Trucated images
    height = 24
    width = 24
    distorted_image = tf.random_crop(reshaped_image, [height, width, 3])
 
    # Randomly distort image
    distorted_image = tf.image.random_flip_left_right(distorted_image) # horizontal
    distorted_image = tf.image.random_brightness(distorted_image,
                                                   max_delta=63)  # brightness
    distorted_image = tf.image.random_contrast(distorted_image,
                                                 lower=0.2, 
                                                 upper=1.8) # contrast
    # Standardization
    float_image = tf.image.per_image_standardization(distorted_image)

    float_image.set_shape([height, width, 3])
      
    label_decoded.set_shape([1])

    batch_size = 128
    
    # To make sure the shuttfle process is substantial
    min_fraction_of_examples_in_queue = 0.4
    min_queue_examples = int(train_size*min_fraction_of_examples_in_queue)
    batch_pixel, batch_label = tf.train.shuffle_batch([float_image, label_decoded],
                                                   batch_size=batch_size,
                                                   num_threads=16,
                                                   capacity=min_queue_examples + 3 * batch_size,
                                                   min_after_dequeue=min_queue_examples)
    batch_label = tf.reshape(batch_label, [batch_size])
  return batch_pixel, batch_label

### Load data for training

In [0]:
def train_data():
  if not data_dir:
    raise ValueError('Please supply a data_dir')
  new_data_dir = os.path.join(data_dir, 'cifar-10-batches-bin')
  
  filenames = [os.path.join(new_data_dir, 'data_batch_%d.bin' % i)
               for i in range(1, 6)] 
  for f in filenames:
    if not tf.gfile.Exists(f):
      raise ValueError('Failed to find file: ' + f)
      
  filename_queue = tf.train.string_input_producer(filenames)
  
  distorted_image, label = distort_data(filename_queue)
  
  return distorted_image, label

### Define model: Logistic Regression

In [0]:
# build logistic regression model
def inference_lg(images):
  # images: returned from distorted_inputs_model
  
  # logits
  with tf.variable_scope('logits') as scope:
    reshape = tf.reshape(images, [images.get_shape().as_list()[0], -1])
    dim = reshape.get_shape()[1].value
    weights = _variable_with_weight_decay('weights', 
                                          shape = [dim, num_label],
                                          stddev=0.04, wd=0.04)
    biases = tf.get_variable('biases', [num_label], dtype=tf.float32,
                             initializer=tf.constant_initializer(0.0))
    softmax_linear = tf.add(tf.matmul(reshape, weights), biases, name=scope.name)
    _activation_summary(softmax_linear)

  return softmax_linear

### Define model: CNN

In [0]:
# build CNN model with Max-Pooling
def inference_cnn(images):
  
  # convolutional 1: 
  with tf.variable_scope('conv1') as scope:
    kernel = _variable_with_weight_decay('weights',
                                         shape=[5, 5, 3, 64],
                                         stddev=5e-2,
                                         wd=None)   
    conv = tf.nn.conv2d(images, kernel, [1, 1, 1, 1], padding='SAME')
    biases = tf.get_variable('biases', [64], 
                             tf.float32,
                             tf.constant_initializer(0.0))
    pre_activation = tf.nn.bias_add(conv, biases)
    conv1 = tf.nn.relu(pre_activation, name=scope.name)
    _activation_summary(conv1)
    
  # pooling 1
  ## max pooling:
  pool1 = tf.nn.max_pool(conv1, 
                         ksize=[1, 2, 2, 1], 
                         strides=[1, 2, 2, 1],
                         padding='SAME',
                         name='pool1')    
  
  ## fractional max pooling 1:
  #pool1 = tf.nn.fractional_max_pool(conv1,
  #                                  pooling_ratio=[1.0, 1.44, 1.44, 1.0],
  #                                  name = 'pool1')
   
  ## average pooling 1
  #pool1 = tf.nn.avg_pool(conv1,
  #                      ksize=[1,2,2,1],
  #                      strides=[1,2,2,1],
  #                      padding='SAME',
  #                      name='pool1')
  
  
  # batch normalization 1
  norm1 = tf.nn.lrn(pool1, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75,
                    name='norm1')

  # convolutional 2: 
  with tf.variable_scope('conv2') as scope:
    kernel = _variable_with_weight_decay('weights',
                                         shape=[5, 5, 64, 64],
                                         stddev=5e-2,
                                         wd=None)
    conv2 = tf.nn.conv2d(norm1, kernel, [1, 1, 1, 1], padding='SAME')
    biases = tf.get_variable('biases', [64], 
                              tf.float32,
                              tf.constant_initializer(0.1))
    pre_activation = tf.nn.bias_add(conv, biases)
    conv2 = tf.nn.relu(pre_activation, name=scope.name)
    _activation_summary(conv2)

  # batch normalization 2
  norm2 = tf.nn.lrn(conv2, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75,
                    name='norm2')
  # pooling 2:
  ## max pooling 2:
  #pool2 = tf.nn.max_pool(norm2, 
  #                       ksize=[1, 2, 2, 1],
  #                       strides=[1, 2, 2, 1], 
  #                       padding='SAME',
  #                       name='pool2')
  
  #fractional max pooling: 
  pool2 = tf.nn.fractional_max_pool(norm2,
                                    pooling_ratio=[1.0, 1.44, 1.44, 1.0],
                                    name = 'pool1')
  
  ## average pooling 2
  #pool2 = tf.nn.avg_pool(conv2,
  #                      ksize=[1,2,2,1],
  #                      strides=[1,2,2,1],
  #                      name='pool2')
  
  # fully connected 3
  with tf.variable_scope('local3') as scope:
    reshape = tf.reshape(pool2.output, [images.get_shape().as_list()[0], -1])
    dim = reshape.get_shape()[1].value
    weights = _variable_with_weight_decay('weights', shape=[dim, 384],
                                          stddev=0.04, wd=0.004)
    biases = tf.get_variable('biases', [384], 
                             tf.float32,
                             tf.constant_initializer(0.1))
    local3 = tf.nn.relu(tf.matmul(reshape, weights) + biases, name=scope.name)
    _activation_summary(local3)

  # fully connected 4
  with tf.variable_scope('local4') as scope:
    weights = _variable_with_weight_decay('weights', shape=[384, 192],
                                          stddev=0.04, wd=0.004)
    biases = tf.get_variable('biases', [192], 
                              tf.float32,
                              tf.constant_initializer(0.1))
    local4 = tf.nn.relu(tf.matmul(local3, weights) + biases, name=scope.name)
    _activation_summary(local4)

  # fully connected to logit
  with tf.variable_scope('softmax_linear') as scope:
    weights = _variable_with_weight_decay('weights', [192, num_label],
                                          stddev=1/192.0, wd=None)
    biases = tf.get_variable('biases', num_label,
                              tf.float32,
                              tf.constant_initializer(0.0))
    softmax_linear = tf.add(tf.matmul(local4, weights), biases, name=scope.name)
    _activation_summary(softmax_linear)

  return softmax_linear


### Define loss

In [0]:
def get_loss(logits, labels):
  #logist and labels are returned from inference function
  labels = tf.cast(labels, tf.int64)
  cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
      labels=labels, logits=logits, name='cross_entropy_per_example')
  cross_entropy_mean = tf.reduce_mean(cross_entropy, name='cross_entropy')
  tf.add_to_collection('losses', cross_entropy_mean)
  return tf.add_n(tf.get_collection('losses'), name='total_loss')

In [0]:
def _add_loss_summaries(total_loss):
  #total_loss is returned from get_loss
  # Compute the moving average of all individual losses and the total loss.
  loss_averages = tf.train.ExponentialMovingAverage(0.9, name='avg')
  losses = tf.get_collection('losses')
  loss_averages_op = loss_averages.apply(losses + [total_loss])

  # Attach a scalar summary to all individual losses and the total loss
  for l in losses + [total_loss]:
    # Name each loss as '(raw)' and name the moving average version of the loss
    # as the original loss name.
    tf.summary.scalar(l.op.name + ' (raw)', l)
    tf.summary.scalar(l.op.name, loss_averages.average(l))

  return loss_averages_op

### Initialization of variables

In [0]:
def _variable_with_weight_decay(name, shape, stddev, wd):
    # name: name of the variable
    # shape: list of ints indicatin dimension of parameters
    # stddev: standard deviation of a truncated Gaussian
    # wd: add L2Loss weight decay multiplied by this float. If None, weight
    #    decay is not added for this Variable
  init = tf.truncated_normal_initializer(stddev=stddev, dtype=tf.float32)  
  var = tf.get_variable(name, shape, initializer=init, dtype = tf.float32)
  if wd is not None:
    weight_decay = tf.multiply(tf.nn.l2_loss(var), wd, name='weight_loss')
    tf.add_to_collection('losses', weight_decay)
  return var

### Optimizer

In [0]:
def optimizer(total_loss, global_step):
    # total_loss: returned from get_loss().
    # global_step: Integer Variable counting the number of training steps processed.

  # variables that affect learning rate
  num_batches_per_epoch = train_size / batch_size
  decay_steps = int(num_batches_per_epoch * 350)

  # decay the learning rate exponentially based on the number of steps.
  # decayed_learning_rate = learning_rate *decay_rate ^ (global_step / decay_steps)
  lr = tf.train.exponential_decay(0.1,
                                  global_step,
                                  decay_steps,
                                  0.1,
                                  staircase=True)
  tf.summary.scalar('learning_rate', lr)

  # generate moving averages of all losses and associated summaries
  loss_averages_op = _add_loss_summaries(total_loss)

  # compute gradients.
  with tf.control_dependencies([total_loss]):
       opt = tf.train.GradientDescentOptimizer(lr)
       grads = opt.compute_gradients(total_loss)

  # apply gradients
  apply_gradient_op = opt.apply_gradients(grads, global_step=global_step)

  # add histograms for trainable variables.
  for var in tf.trainable_variables():
    tf.summary.histogram(var.op.name, var)

  # add histograms for gradients.
  for grad, var in grads:
    if grad is not None:
      tf.summary.histogram(var.op.name + '/gradients', grad)

  # track the moving averages of all trainable variables.
  variable_averages = tf.train.ExponentialMovingAverage(0.9999, global_step)
  with tf.control_dependencies([apply_gradient_op]):
    variables_averages_op = variable_averages.apply(tf.trainable_variables())

  return variables_averages_op

### Train model

In [0]:
def train(model_type):
  with tf.Graph().as_default():
    global_step = tf.train.get_or_create_global_step()

    # get images and labels for CIFAR-10.
    # force input pipeline to CPU:0 to avoid operations sometimes ending up on
    # GPU and resulting in a slow down.
    images_train, labels_train = train_data()

    # computes the logits predictions from the
    logits_train = model_type(images_train)
    
    # train loss.
    loss_train = get_loss(logits_train, labels_train)
    tf.summary.scalar('Loss', loss_train)

    
    train_op = optimizer(loss_train, global_step)

    class _LoggerHook(tf.train.SessionRunHook):
      """Logs loss and runtime."""

      def begin(self):
        self._step = -1
        self._start_time = time.time()

      def before_run(self, run_context):
        self._step += 1
        return tf.train.SessionRunArgs(loss_train)  # Asks for loss value.

      def after_run(self, run_context, run_values):
        freq = 100
        if self._step % freq == 0:
          current_time = time.time()
          duration = current_time - self._start_time
          self._start_time = current_time

          loss_value = run_values.results
          examples_per_sec = freq * batch_size / duration
          sec_per_batch = float(duration / freq)

          format_str = ('%s: step %d, loss = %.2f (%.1f examples/sec; %.3f '
                        'sec/batch)')
          print (format_str % (datetime.now(), self._step, loss_value,
                               examples_per_sec, sec_per_batch))

    with tf.train.MonitoredTrainingSession(checkpoint_dir=train_dir,
               hooks=[tf.train.StopAtStepHook(last_step=max_step_train),
                      tf.train.NanTensorHook(loss_train),
                       _LoggerHook()],
        config=tf.ConfigProto(log_device_placement=False)) as mon_sess:
      while not mon_sess.should_stop():
        mon_sess.run(train_op)
    
        

### Define summary for tensorboard

In [0]:
def _activation_summary(x):
  #x: Tensor
  tensor_name =  x.op.name
  tf.summary.histogram(tensor_name + '/activations', x)
  tf.summary.scalar(tensor_name + '/sparsity',
                                       tf.nn.zero_fraction(x))

### Execution of training

In [0]:
if tf.gfile.Exists(train_dir):
   tf.gfile.DeleteRecursively(train_dir)
   tf.gfile.MakeDirs(train_dir)  

In [0]:
train(inference_cnn)

INFO:tensorflow:Summary name local3/weight_loss (raw) is illegal; using local3/weight_loss__raw_ instead.
INFO:tensorflow:Summary name local4/weight_loss (raw) is illegal; using local4/weight_loss__raw_ instead.
INFO:tensorflow:Summary name cross_entropy (raw) is illegal; using cross_entropy__raw_ instead.
INFO:tensorflow:Summary name total_loss (raw) is illegal; using total_loss__raw_ instead.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/cifar10_train/model.ckpt.
2018-12-17 07:24:12.846361: step 0, loss = 18.07 (472.2 examples/sec; 0.271 sec/batch)
INFO:tensorflow:global_step/sec: 4.0595
2018-12-17 07:24:37.514020: step 100, loss = 16.49 (518.9 examples/sec; 0.247 sec/batch)
INFO:tensorflow:global_step/sec: 3.98817
2018-12-17 07:25:02.565081: step 200, loss = 15.15 (511.0 examples/sec; 0.251 sec/batch)
INFO:tensorflow

### Tensorboard

In [0]:
# training
LOG_DIR = train_dir
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
# Download and unzip ngrok - you will only need to do this once per session
! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip ngrok-stable-linux-amd64.zip

# Launch the ngrok background process
get_ipython().system_raw('./ngrok http 6006 &')

# Get the public URL and be sorted!
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

### Evaluation

In [0]:
# Load data for evaluation
def eval_data():
  if not data_dir:
    raise ValueError('Please supply a data_dir')
  new_data_dir = os.path.join(data_dir, 'cifar-10-batches-bin')
  
  filenames = [os.path.join(new_data_dir, 'test_batch.bin')]
  for f in filenames:
      if not tf.gfile.Exists(f):
        raise ValueError('Failed to find file: ' + f)
        
  filename_queue = tf.train.string_input_producer(filenames)
  
  distorted_image, label = distort_data(filename_queue)
  
  return distorted_image, label

In [0]:
#evaluation on the whole set
def evaluate(saver, summary_writer, top_k_op, summary_op):
  with tf.Session() as sess:
    ckpt = tf.train.get_checkpoint_state(train_dir)
    if ckpt and ckpt.model_checkpoint_path:
      saver.restore(sess, ckpt.model_checkpoint_path)
      global_step = ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1]
    else:
      print('No checkpoint file found')
      return

    # Start the queue runners.
    coord = tf.train.Coordinator()
    try:
      threads = []
      for qr in tf.get_collection(tf.GraphKeys.QUEUE_RUNNERS):
        threads.extend(qr.create_threads(sess, coord=coord, daemon=True,
                                         start=True))

      num_iter = int(math.ceil(test_size / batch_size))
      true_count = 0  # Counts the number of correct predictions.
      total_sample_count = num_iter * batch_size
      
      step = 0
      while step < num_iter and not coord.should_stop():
         predictions = sess.run([top_k_op])
         true_count += np.sum(predictions)
         step += 1

      # Compute precision @ 1.
      precision = true_count / total_sample_count
      print('%s: precision @ 1 = %.3f' % (datetime.now(), precision))

      summary = tf.Summary()
      summary.ParseFromString(sess.run(summary_op))
      summary.value.add(tag='Precision @ 1', simple_value=precision)
      summary_writer.add_summary(summary, global_step)
    except Exception as e:  # pylint: disable=broad-except
      coord.request_stop(e)

    coord.request_stop()
    coord.join(threads, stop_grace_period_secs=10)

In [0]:
def run_evaluate(model_type, run_once):
  with tf.Graph().as_default() as g:
    # Get images and labels for CIFAR-10. 
    images, labels = eval_data()

    # Get loss
    logits = model_type(images)
    loss = get_loss(logits, labels)
    
    # Get predictions.
    top_k_op = tf.nn.in_top_k(logits, labels, 1)

    # Restore the moving average version of the learned variables for eval.
    variable_averages = tf.train.ExponentialMovingAverage(0.9999)
    variables_to_restore = variable_averages.variables_to_restore()
    saver = tf.train.Saver(variables_to_restore)

    # Build the summary operation based on the TF collection of Summaries.
    summary_op = tf.summary.merge_all()

    summary_writer = tf.summary.FileWriter(eval_dir, g)
    
    while True:
      evaluate(saver, summary_writer, top_k_op, summary_op)
      if run_once:
           break
        #time.sleep(FLAGS.eval_interval_secs)

In [0]:
if tf.gfile.Exists(eval_dir):
    tf.gfile.DeleteRecursively(eval_dir)
tf.gfile.MakeDirs(eval_dir)

In [0]:
run_evaluate(inference_cnn, True)